In [11]:
# Tutorial: https://www.youtube.com/watch?v=dRcvJRmqFHQ&lc=UgyUDvoIA0A37EWWoC54AaABAg.9Aem3mXGQzz9Aep5dNrAY6
import requests
from bs4 import BeautifulSoup
import json
import time
import csv
import pandas as pd
import os

class ZillowScraper():
    results = []
    headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'zguid=23|%24d7ce356c-e3c5-4a48-85b3-61d0af33c3a1; zgsession=1|76ac70ba-62bc-4f77-81a1-b46b55a3f268; _ga=GA1.2.656681018.1593695925; _gid=GA1.2.1901059192.1593695925; zjs_user_id=null; zjs_anonymous_id=%22d7ce356c-e3c5-4a48-85b3-61d0af33c3a1%22; _pxvid=930c8230-bc66-11ea-9dd2-0242ac120007; _gcl_au=1.1.2002891296.1593696177; KruxPixel=true; DoubleClickSession=true; _pin_unauth=dWlkPU9EbG1aR0ptTTJVdFkySXpPUzAwWVdZMUxXSTFNVE10TnpFNE1EZGpOamxsWWpZNQ; _fbp=fb.1.1593696180071.236888825; KruxAddition=true; g_state={"i_p":1593716288950,"i_l":1}; JSESSIONID=0C394BB3DC7B91E479B72A328353817A; GASession=true; _pxff_bsco=1; AWSALB=ICqV1IKa+cj8ecut5WcXnUJCq3pP//RbqBzZ49F9UxNm0bLvluUKA3yHj1v2AbnyZ74OnEhFTZXZYd39kdbddeIij3ZzH2d20INkvIhGwFyGR+evOyhe+uUmhlil; AWSALBCORS=ICqV1IKa+cj8ecut5WcXnUJCq3pP//RbqBzZ49F9UxNm0bLvluUKA3yHj1v2AbnyZ74OnEhFTZXZYd39kdbddeIij3ZzH2d20INkvIhGwFyGR+evOyhe+uUmhlil; search=6|1596363321838%7Crect%3D43.45764218649565%252C-109.08936284375%252C30.85123360186431%252C-129.52393315625%26rid%3D9%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26days%3D90%26type%3Dhouse%26lt%3Dfsba%252Cfsbo%252Cfore%252Cnew%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%099%09%09%09%09%09%09; _px3=d4336234c4010c751aebc89c46325b0fb974e47215ce19445dfb759a629e81dc:X7e2f9pB1glpxKBrZ31UbYr/SXJFF+UxNWu+hiMsI+DQ7Mqle+9ez9SpkHaEa0n8KjNaAtb3oFQCfBoM9SkdFQ==:1000:AfuabQEQyo2l//8QKend7zOjb5sDW5Is+c6zQcm/upUYyYJc1jNQiyYnwroZOcBIjJBNJ8+vUKEJ3TlBoNaX/O7UbA0YkImLQb+VkAlAZWRj+w8lmqxZjshBeq6rrIg7GVsOSD+a1N8QtabSrjTIOF1SbQdGaZFzwVUu+aT6YiQ=; _uetsid=b093ce5e-93c4-0334-9c39-42ee7af2a929; _uetvid=e59e74ae-427f-984f-1765-cdc49ba1fa47',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    }
        
    def fetch(self, url, params):
        response = requests.get(url, headers=self.headers, params=params)
        print(response.status_code)
        return response
    
    def parse(self, response):
        content = BeautifulSoup(response, 'lxml')
        deck = content.find('ul', {'class': 'photo-cards photo-cards_wow photo-cards_short'})
        for card in deck.contents:
            script = card.find('script', {'type': 'application/ld+json'})
            if script:
                script_json = json.loads(script.contents[0])
                print(script_json['geo'])
                try:
                    latitude = script_json['geo']['latitude']
                    longitude = script_json['geo']['longitude']
                except KeyError:
                    latitude = 'null'
                    longitude = 'null'
                except AttributeError:
                    latitude = 'null'
                    longitude = 'null'
                loop_response = requests.get(script_json['url'], headers=self.headers)
                loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
                try:
                    loop_result2 = loop_soup.findAll('div', class_="ds-expandable-card-section-default-padding")[1].findAll('li', class_="ds-home-fact-list-item")
                except IndexError:
                    loop_result2 = 'null'
                lot = 'null';
                house_type = 'null';
                year_build = 'null';
                parking = 'null';
                hoa = 'null';
                lot_conversion_to_sqft = 'null';
                zillow_price_per_sqft = 'null';
                
                if loop_result2:
                    for contents in loop_result2:
                        print(contents.text)
                        if "Lot" in contents.text.split(":")[0]:
                            lot = contents.text.split(":")[1];
                        if "Type" in contents.text.split(":")[0]:
                            house_type = contents.text.split(":")[1];
                        if "Year built" in contents.text.split(":")[0]:
                            year_build = contents.text.split(":")[1];
                        if "Parking" in contents.text.split(":")[0]:
                            parking = contents.text.split(":")[1];
                        if "HOA" in contents.text.split(":")[0]:
                            hoa = contents.text.split(":")[1];
                        if "Price/sqft" in contents.text.split(":")[0]:
                            zillow_price_per_sqft = contents.text.split(":")[1];
                
                    try:
                        zestimate = loop_soup.findAll('div', class_="ds-expandable-card-section-default-padding")[2].find('p', class_="Text-aiai24-0 sc-fzoxKX sc-oTpqt loFLRQ").text
                        rent_zestimate = loop_soup.findAll('div', class_="ds-expandable-card-section-default-padding")[4].find('p', class_="Text-aiai24-0 sc-fzoxKX sc-oTpqt loFLRQ").text
                    except AttributeError:
                        zestimate = 'null'
                        rent_zestimate = 'null'
                    try:
                        if (str(lot.split(" ")[1]) == "sqft"):
                            lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                        elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
                            lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                        elif (lot == "No Data"):
                            lot_conversion_to_sqft = 'null'
                        else:
                            lot_conversion_to_sqft = 'null'
                    except IndexError:
                        lot_conversion_to_sqft = 'null'
                        
                    self.results.append({
                        'Listing_type': card.find('div', {'class': 'list-card-type'}).text,
                        'Price': card.find('div', {'class': 'list-card-price'}).text,
                        'Full_address': script_json['name'],
                        'Street': script_json['address']['streetAddress'],
                        'City': script_json['address']['addressLocality'],
                        'State': script_json['address']['addressRegion'],
                        'Zip_code': script_json['address']['postalCode'],
                        'Latitude': latitude,
                        'Longitude': longitude,
                        'Square_Feet': script_json['floorSize']['value'],
                        'Lot': lot,
                        'Lot_conversion_to_sqft': lot_conversion_to_sqft,
                        'House_type': house_type,
                        'Year_build': year_build,
                        'Parking': parking,
                        'Hoa': hoa,
                        'Zillow_price_per_sqft': zillow_price_per_sqft,
                        'Zestimate': zestimate,
                        'Rent_zestimate': rent_zestimate,
                        'House_website': script_json['url']
                    })
                    time.sleep(3)
                
    def to_csv(self):
        dirName = 'Output/Zillow/'
        if not os.path.exists(dirName):
            os.mkdir(dirName)
            print("Directory ", dirName, " Folder created ")
        else:
            print("Directory ", dirName, " Folder already exists")
            
        with open('./' + dirName + 'Q3-2020_Multi_Family_Data_page17-20.csv', 'w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys())
            writer.writeheader()
            
            for row in self.results:
                writer.writerow(row)

    def to_dftable(self):
        house_data = pd.DataFrame(self.results)
        print(house_data)

#     Filter Out Coming Soon, Auction, Home Type: Houses, Days On Zillow: 90 Days
    def run(self):
        url = 'https://www.zillow.com/ca/duplex/'
        for page in range(17,21):
            params = {
                'searchQueryState':'{"pagination":{"currentPage":%s},"usersSearchTerm":"ca","mapBounds":{"west":-130.93471906436687,"east":-107.64370343936687,"south":31.980790552461904,"north":48.0418909706512},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false},"isCondo":{"value":false}},"isListVisible":true}' %page
            }
            res = self.fetch(url, params)
            self.parse(res.text)
            time.sleep(2)
        self.to_dftable()
        self.to_csv()
        
if __name__ == '__main__':
    scraper = ZillowScraper()
    scraper.run();

200
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 33.912672, 'longitude': -118.359014}
Type:Multiple Occupancy
Year built:1940
Heating:Stove, Gas
Cooling:Central
Parking:3 spaces
Price/sqft:$333
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 34.471309, 'longitude': -117.319743}
Type:Multiple Occupancy
Year built:1984
Heating:Other
Cooling:Central
Parking:2 spaces
Price/sqft:$126
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 35.356238, 'longitude': -120.845702}
Type:Multiple Occupancy
Year built:1962
Heating:Electric, Gas
Cooling:None
Parking:3 spaces
Price/sqft:$625
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 38.612645, 'longitude': -121.431774}
Type:Multiple Occupancy
Year built:1956
Heating:Forced air, Electric
Cooling:Central
Parking:2 spaces
Price/sqft:$366
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 37.802691, 'longitude': -122.438744}
Type:Mult

{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 39.04249, 'longitude': -122.91647}
Type:Multiple Occupancy
Year built:1888
Heating:Electric
Cooling:No Data
Parking:2 spaces
Price/sqft:$197
200
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 34.077018, 'longitude': -118.293073}
Type:Multiple Occupancy
Year built:1940
Heating:Forced air
Cooling:Central
Parking:20 spaces
Price/sqft:$416
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 34.133154, 'longitude': -118.188175}
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 36.340632, 'longitude': -119.312446}
Type:Multiple Occupancy
Year built:1960
Heating:Wall
Cooling:Evaporative
Parking:2 spaces
Price/sqft:$159
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 38.591542, 'longitude': -121.395858}
Type:Multiple Occupancy
Year built:1962
Heating:Forced air
Cooling:Central
Parking:2 spaces
Price/sqft:$221
{'@type': 'GeoCoord

{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 41.529254, 'longitude': -120.169488}
Type:Multiple Occupancy
Year built:1915
Heating:No Data
Cooling:No Data
Parking:No Data
Price/sqft:$82
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 38.480648, 'longitude': -121.437767}
Type:Multiple Occupancy
Year built:1969
Heating:Forced air
Cooling:Central
Parking:3 spaces
Price/sqft:$173
200
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 32.777904, 'longitude': -117.252134}
Type:Multiple Occupancy
Year built:1937
Heating:Forced air, Gas
Cooling:Wall
Parking:3 spaces
Price/sqft:$1,040
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 33.791877, 'longitude': -118.164715}
Type:Multiple Occupancy
Year built:1923
Heating:Wall
Cooling:None
Parking:No Data
Price/sqft:$387
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 34.16494, 'longitude': -117.292068}
Type:Multiple Occupancy
Ye

Type:Multiple Occupancy
Year built:1925
Heating:Baseboard, Gas
Cooling:Other
Parking:No Data
Price/sqft:$110
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 32.815086, 'longitude': -117.25173}
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 37.954788, 'longitude': -121.268986}
Type:Multiple Occupancy
Year built:2020
Heating:Forced air, Electric
Cooling:Central
Parking:2 spaces
Price/sqft:$506
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 36.697811, 'longitude': -119.557625}
Type:Multiple Occupancy
Year built:1939
Heating:No Data
Cooling:Evaporative, Wall
Parking:No Data
Price/sqft:$157
200
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 34.005322, 'longitude': -118.317879}
Type:Multiple Occupancy
Year built:1937
Heating:Other, Gas
Cooling:Central, Other
Parking:3 spaces
Price/sqft:$405
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 33.652419, 'longitude': -117

{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 35.387543, 'longitude': -119.165845}
Type:Multiple Occupancy
Year built:2004
Heating:No Data
Cooling:No Data
Parking:5 spaces
Price/sqft:$181
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 37.290849, 'longitude': -120.475842}
Type:Multiple Occupancy
Year built:1960
Heating:Wall, Electric, Gas
Cooling:Evaporative
Parking:4 spaces
Price/sqft:$147
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 37.396604, 'longitude': -122.06374}
Type:Multiple Occupancy
Year built:2020
Heating:Forced air, Gas
Cooling:Central, Wall
Parking:No Data
Price/sqft:$880
{'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 34.146229, 'longitude': -117.997755}
Type:Multiple Occupancy
Year built:1907
Heating:Wall, Electric, Gas
Cooling:Wall
Parking:1 space
Price/sqft:$418
                   Listing_type        Price  \
0    Multi-family home for sale     $877,000   
1    Mul